In [94]:
import urllib.request
import urllib.parse
import urllib.error
import ssl
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
import pandas as pd
import re
from datetime import datetime

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [42]:
# extract information of houses on market in Fremont, 40 houses
market_houses = []
markethouse_url = 'https://www.zillow.com/fremont-ca/houses/?searchQueryState={%22pagination%22:{},%22mapBounds%22:{%22west%22:-122.24536320019531,%22east%22:-121.77157779980469,%22south%22:37.37488741627461,%22north%22:37.68364018147819},%22regionSelection%22:[{%22regionId%22:11540,%22regionType%22:6}],%22isMapVisible%22:true,%22mapZoom%22:11,%22filterState%22:{%22sortSelection%22:{%22value%22:%22globalrelevanceex%22},%22isCondo%22:{%22value%22:false},%22isMultiFamily%22:{%22value%22:false},%22isManufactured%22:{%22value%22:false},%22isLotLand%22:{%22value%22:false},%22isTownhouse%22:{%22value%22:false},%22isApartment%22:{%22value%22:false}},%22isListVisible%22:true}'
market_req = Request(markethouse_url, headers = {'user-Agent':'Mozilla/5.0'})
market_text = urlopen(market_req).read().decode(encoding="utf-8", errors='ignore')
market_soup = BS(market_text, 'html.parser')
for link in market_soup.findAll('a',attrs = {'class':'list-card-link', 'tabindex':"0"}, href = True):
    market_houses.append(link['href'])


In [43]:
len(market_houses)

40

In [60]:
#create a empty dataframe
house_df = pd.DataFrame()

In [5]:
#informations on the first listing page of all 40 houses
bds = []
bath = []
areas = []
price_ls = []
adr_ls = []


req = Request(markethouse_url, headers = {'user-Agent':'Mozilla/5.0'})
text = urlopen(req).read().decode(encoding="utf8", errors='ignore')
soup = BS(text, 'html.parser')
    
house_prices = soup.findAll('div', attrs = {'class':'list-card-price'})
for price in house_prices:
    price_ls.append(price.get_text())


    
#extract room info
lists = soup.findAll('ul', attrs = {'class':'list-card-details'})
details = soup.findAll('ul', attrs = {'class':'list-card-details'})
#print(len(details))
for detail in details:
    #print(detail.li)
    bds.append(detail.li.get_text())
    bath.append(detail.li.next_sibling.get_text())
    areas.append(detail.li.next_sibling.next_sibling.get_text())
    
#extract address info
addresses = soup.findAll('address', attrs = {'class':'list-card-addr'})
for address in addresses:
    adr_ls.append(address.get_text())

In [45]:
len(bds)

40

In [56]:
#details after opening each url of a house

lotsize = []
year = []
ac = []
heat = []
park = []
price_sqft = []
hoa = []

elem = []
mid =[]
high=[]

elem_s=[]
mid_s=[]
high_s=[]

for url in market_houses:
    req = Request(url, headers = {'user-Agent':'Mozilla/5.0'})
    text = urlopen(req).read().decode(encoding="utf-8", errors='ignore')
    soup = BS(text, 'html.parser')
    
    #iterate all house url on first page, and scrape the lot_size, year, ac, heater, garage, price/sqft info
    home_values = soup.findAll('span', attrs = {'class':'ds-body ds-home-fact-value'})
    hv_ls = []
    for value in home_values:
        hv_ls.append(value.get_text())   
    #print(hv_ls)
    if len(hv_ls) == 0:
        hv_ls = ['None']*8
    elif len(hv_ls) == 8:
        lotsize.append(hv_ls[-2])
        hoa.append(hv_ls[5])
        year.append(hv_ls[1])
        ac.append(hv_ls[3])
        heat.append(hv_ls[2])
        park.append(hv_ls[4])
        price_sqft.append(hv_ls[-1])
    else:
        lotsize.append(hv_ls[-2])
        hoa.append('None')
        year.append(hv_ls[1])
        ac.append(hv_ls[3])
        heat.append(hv_ls[2])
        park.append(hv_ls[4])
        price_sqft.append(hv_ls[-1])

    #scrape school info    
    schools = soup.findAll('div', attrs = {'class':'ds-nearby-schools-list'})
    if len(schools) == 0:
        scl = ['None', 'None', 'None']
        scl_score = ['None', 'None', 'None']
    elif len(list(schools[0].children)) == 0:
        scl = ['None', 'None', 'None']
        scl_score = ['None', 'None', 'None']
    else:
        scl = ['None', 'None', 'None']
        scl_score = ['None', 'None', 'None']
        for school in schools[0].children:  
            #print(school.span.get_text())
            grade = school.findAll('span')[3].get_text()
            if grade == 'K-6' or grade == 'K-5' or grade == 'K-8':
                scl[0] = school.a.get_text()
                scl_score[0] = school.span.get_text()
            if grade == '7-8' or grade == '6-8':
                scl[1] = school.a.get_text()
                scl_score[1] = school.span.get_text()
            if grade == '9-12':
                scl[2] = school.a.get_text()
                scl_score[2] = school.span.get_text()
    elem.append(scl[0])
    mid.append(scl[1])
    high.append(scl[2])
    elem_s.append(scl_score[0])
    mid_s.append(scl_score[1])
    high_s.append(scl_score[2])


['Single Family', '1971', 'Forced air', 'Central', '2 spaces', '5,662 sqft', '$574']
['Single Family', '1967', 'Forced air', 'None', 'Attached Garage', '5,797 sqft', '$620']
['Single Family', '1977', 'Forced air', 'Central', '2 spaces', '5,479 sqft', '$624']
['Single Family', '1992', 'Forced air', 'Central', '2 spaces', '$40/month', '3,598 sqft', '$783']
['Single Family', '1923', 'Wall', 'Wall', '2 spaces', '5,039 sqft', '$828']
['Single Family', '1945', 'Forced air', 'Central', 'Attached Garage', 'None', '5,898 sqft', '$853']
['Single Family', '1986', 'Forced air', 'Central', '2 spaces', '7,688 sqft', '$695']
['Single Family', '1978', 'Forced air', 'None', '2 spaces', '$29/month', '1,973 sqft', '$553']
['Single Family', '1990', 'Forced air', 'Central', '2 spaces', '9,578 sqft', '$492']
['Single Family', '2014', 'Forced air', 'Central', '2 spaces', '$150/month', '2,225 sqft', '$712']
['Single Family', '2010', 'Forced air', 'Central', '2 spaces', '$160/month', '4,534 sqft', '$556']
['Si

In [57]:
len(hoa)

40

In [61]:
#add a columns to the empty dataframe
house_df['bds']=bds
house_df['bath']=bath
house_df['areas']=areas
house_df['lot_size']=lotsize
house_df['year_built']=year
house_df['a/c']=ac
house_df['heater']=heat
house_df['garage']=park
house_df['price/sqft']=price_sqft
house_df['elem_score']=elem_s
house_df['middle_score']=mid_s
house_df['high_score']=high_s
house_df['elem_school'] = elem
house_df['middle_school'] = mid
house_df['high_school'] = high
house_df['price'] = price_ls
house_df['HOA'] = hoa
house_df['address']=adr_ls

In [62]:
house_df.head()

,bds,bath,areas,lot_size,year_built,a/c,heater,garage,price/sqft,elem_score,middle_score,high_score,elem_school,middle_school,high_school,price,HOA,address
0,4 bds,2 ba,"1,830 sqft","5,662 sqft",1971,Central,Forced air,2 spaces,$574,7,8,10,Brookvale Elementary School,Thornton Junior High School,American High School,"$1,050,000",None,"35465 Collier Pl, Fremont, CA 94536"
1,3 bds,2 ba,"1,612 sqft","5,797 sqft",1967,None,Forced air,Attached Garage,$620,7,8,10,Brookvale Elementary School,Thornton Junior High School,American High School,"$999,888",None,"35462 Ronda Ct, Fremont, CA 94536"
2,3 bds,3 ba,"1,440 sqft","5,479 sqft",1977,Central,Forced air,2 spaces,$624,6,5,7,Brier Elementary School,G. M. Walters Junior High School,John F. Kennedy High School,"$898,000",None,"4730 Mowry Ave, Fremont, CA 94538"
3,3 bds,3 ba,"1,593 sqft","3,598 sqft",1992,Central,Forced air,2 spaces,$783,8,8,10,Ardenwood Elementary School,Thornton Junior High School,American High School,"$1,248,000",$40/month,"34327 Eucalyptus Ter, Fremont, CA 94555"
4,2 bds,1 ba,760 sqft,"5,039 sqft",1923,Wall,Wall,2 spaces,$828,8,7,8,Niles Elementary School,Centerville Junior High,Washington High School,"$629,000",None,"361 Riverside Ave, Fremont, CA 94536"


In [12]:
#find all links of 80 sold houses on the first 2 page of the search, sorted by newest to oldest
soldpage_urls = ['https://www.zillow.com/fremont-ca/sold/house_type/?searchQueryState={%22pagination%22:{},%22mapBounds%22:{%22west%22:-122.24536320019531,%22east%22:-121.77157779980469,%22south%22:37.37488741627461,%22north%22:37.68364018147819},%22regionSelection%22:[{%22regionId%22:11540,%22regionType%22:6}],%22mapZoom%22:11,%22filterState%22:{%22isCondo%22:{%22value%22:false},%22isMultiFamily%22:{%22value%22:false},%22isManufactured%22:{%22value%22:false},%22isLotLand%22:{%22value%22:false},%22isTownhouse%22:{%22value%22:false},%22isApartment%22:{%22value%22:false},%22isRecentlySold%22:{%22value%22:true},%22isForSaleByAgent%22:{%22value%22:false},%22isForSaleByOwner%22:{%22value%22:false},%22isNewConstruction%22:{%22value%22:false},%22isComingSoon%22:{%22value%22:false},%22isAuction%22:{%22value%22:false},%22isForSaleForeclosure%22:{%22value%22:false},%22isPreMarketForeclosure%22:{%22value%22:false},%22isPreMarketPreForeclosure%22:{%22value%22:false}},%22isListVisible%22:true,%22isMapVisible%22:false}',
                 'https://www.zillow.com/fremont-ca/sold/house_type/2_p/?searchQueryState={%22pagination%22:{%22currentPage%22:2},%22mapBounds%22:{%22west%22:-122.24536320019531,%22east%22:-121.77157779980469,%22south%22:37.37488741627461,%22north%22:37.68364018147819},%22regionSelection%22:[{%22regionId%22:11540,%22regionType%22:6}],%22mapZoom%22:11,%22filterState%22:{%22isCondo%22:{%22value%22:false},%22isMultiFamily%22:{%22value%22:false},%22isManufactured%22:{%22value%22:false},%22isLotLand%22:{%22value%22:false},%22isTownhouse%22:{%22value%22:false},%22isApartment%22:{%22value%22:false},%22isRecentlySold%22:{%22value%22:true},%22isForSaleByAgent%22:{%22value%22:false},%22isForSaleByOwner%22:{%22value%22:false},%22isNewConstruction%22:{%22value%22:false},%22isComingSoon%22:{%22value%22:false},%22isAuction%22:{%22value%22:false},%22isForSaleForeclosure%22:{%22value%22:false},%22isPreMarketForeclosure%22:{%22value%22:false},%22isPreMarketPreForeclosure%22:{%22value%22:false}},%22isListVisible%22:true,%22isMapVisible%22:false}']
sold_urllist = []
for url in soldpage_urls:
    req = Request(url, headers = {'user-Agent':'Mozilla/5.0'})
    text = urlopen(req).read().decode(encoding="utf-8", errors='ignore')
    soup = BS(text, 'html.parser')
    for link in soup.findAll('a',attrs = {'class':'list-card-link', 'tabindex':"0"}, href = True):
        
        sold_urllist.append(link['href'])

In [13]:
len(sold_urllist[40:80])

40

In [14]:
soldhouse_df = pd.DataFrame()

In [15]:
#informations on the first listing page of all 40 houses
bds_sold = []
bath_sold = []
areas_sold = []
price_ls_sold = []
adr_ls_sold = []
sold_dates = []
for url in soldpage_urls:
    req_sold = Request(url, headers = {'user-Agent':'Mozilla/5.0'})
    text_sold = urlopen(req_sold).read().decode(encoding="utf8", errors='ignore')
    soup_sold = BS(text_sold, 'html.parser')
    
    soldhouse_prices = soup_sold.findAll('div', attrs = {'class':'list-card-price'})
    for price in soldhouse_prices:
        price_ls_sold.append(price.get_text())


    
    #extract room info
    lists_sold = soup_sold.findAll('ul', attrs = {'class':'list-card-details'})
    details_sold = soup_sold.findAll('ul', attrs = {'class':'list-card-details'})
    #print(len(details))
    for detail in details_sold:
        #print(detail.li)
        bds_sold.append(detail.li.get_text())
        bath_sold.append(detail.li.next_sibling.get_text())
        areas_sold.append(detail.li.next_sibling.next_sibling.get_text())
    
    #extract address info
    addresses_sold = soup_sold.findAll('address', attrs = {'class':'list-card-addr'})
    for address in addresses_sold:
        adr_ls_sold.append(address.get_text())
        
    #extract sold dates
    dates = soup_sold.findAll('div', attrs={'class':'list-card-variable-text list-card-img-overlay'})
    for date in dates:
        sold_dates.append(date.get_text().split(' ')[1])

In [16]:
#details after opening each url of a house
lotsize_sold1 = []
year_sold1 = []
ac_sold1 = []
heat_sold1 = []
park_sold1 = []

hoa_sold1 = []

elem_sold1 = []
mid_sold1 =[]
high_sold1=[]

elem_s_sold1=[]
mid_s_sold1=[]
high_s_sold1=[]

neighborhood_sold1 = []

for url in sold_urllist[0:40]:
    #print(url)
    req_sold1 = Request(url, headers = {'user-Agent':'Mozilla/5.0'})
    text_sold1 = urlopen(req_sold1).read().decode(encoding="utf-8", errors='ignore')
    soup_sold1 = BS(text_sold1, 'html.parser')
    
    #iterate all house url on first page, and scrape the lot_size, year, ac, heater, garage, price/sqft info
    home_values_sold1 = soup_sold1.findAll('div', attrs = {'class':'fact-value'})
    hv_ls_sold1 = []
    for value in home_values_sold1:
        hv_ls_sold1.append(value.get_text())   
    if len(hv_ls_sold1) == 0:
        hv_ls_sold1 = ['None']*7
    #print(hv_ls_sold1)   
    lotsize_sold1.append(hv_ls_sold1[6])
    hoa_sold1.append(hv_ls_sold1[5])
    year_sold1.append(hv_ls_sold1[1])
    ac_sold1.append(hv_ls_sold1[3])
    heat_sold1.append(hv_ls_sold1[2])
    park_sold1.append(hv_ls_sold1[4])

    

    #scrape school info    
    schools1 = soup_sold1.findAll('li', attrs = {'class':'nearby-school assigned-school clearfix'})
    if len(schools1) == 0:
        scl1 = ['None', 'None', 'None']
        scl_score1 = ['None', 'None', 'None']
    else:
        scl1 = ['None', 'None', 'None']
        scl_score1 = ['None', 'None', 'None']
        for school in schools1:  
            #print(school.span.get_text())
            grade = school.findAll('div')[5].get_text()
            if grade == 'K-6' or grade == 'K-5' or grade == 'K-8':
                scl1[0] = school.a.get_text()
                scl_score1[0] = school.span.get_text()
            if grade == '7-8' or grade == '6-8':
                scl1[1] = school.a.get_text()
                scl_score1[1] = school.span.get_text()
            if grade == '9-12':
                scl1[2] = school.a.get_text()
                scl_score1[2] = school.span.get_text()
    elem_sold1.append(scl1[0])
    mid_sold1.append(scl1[1])
    high_sold1.append(scl1[2])
    elem_s_sold1.append(scl_score1[0])
    mid_s_sold1.append(scl_score1[1])
    high_s_sold1.append(scl_score1[2])
    
    #scrape Neiborhood info
    for nb in soup_sold1.findAll(string=re.compile("^Neighborhood:")):
        neighborhood_sold1.append(nb.split(':')[1])


In [17]:
len(neighborhood_sold1)

40

In [52]:
#details after opening each url of a house
lotsize_sold2 = []
year_sold2 = []
ac_sold2 = []
heat_sold2 = []
park_sold2 = []

hoa_sold2 = []
bath_sold2 = []

elem_sold2 = []
mid_sold2 =[]
high_sold2=[]

elem_s_sold2=[]
mid_s_sold2=[]
high_s_sold2=[]

for url in sold_urllist[40:80]:
    #print(url)
    req_sold2 = Request(url, headers = {'user-Agent':'Mozilla/5.0'})
    text_sold2 = urlopen(req_sold2).read().decode(encoding="utf-8", errors='ignore')
    soup_sold2 = BS(text_sold2, 'html.parser')
    
    #iterate all house url on first page, and scrape the lot_size, year, ac, heater, garage, price/sqft info
    home_values_sold2 = soup_sold2.findAll('div', attrs = {'class':'fact-value'})
    hv_ls_sold2 = []
    for value in home_values_sold2:
        hv_ls_sold2.append(value.get_text())   
    if len(hv_ls_sold2) == 0:
        hv_ls_sold2 = ['None']*7
    #print(hv_ls_sold2)   
    lotsize_sold2.append(hv_ls_sold2[6])
    hoa_sold2.append(hv_ls_sold2[5])
    year_sold2.append(hv_ls_sold2[1])
    ac_sold2.append(hv_ls_sold2[3])
    heat_sold2.append(hv_ls_sold2[2])
    park_sold2.append(hv_ls_sold2[4])

    

    #scrape school info    
    schools2 = soup_sold2.findAll('li', attrs = {'class':'nearby-school assigned-school clearfix'})
    if len(schools2) == 0:
        scl2 = ['None', 'None', 'None']
        scl_score2 = ['None', 'None', 'None']
    else:
        scl2 = ['None', 'None', 'None']
        scl_score2 = ['None', 'None', 'None']
        for school in schools2:  
            #print(school.span.get_text())
            grade = school.findAll('div')[5].get_text()
            if grade == 'K-6' or grade == 'K-5' or grade == 'K-8':
                scl2[0] = school.a.get_text()
                scl_score2[0] = school.span.get_text()
            if grade == '7-8' or grade == '6-8':
                scl2[1] = school.a.get_text()
                scl_score2[1] = school.span.get_text()
            if grade == '9-12':
                scl2[2] = school.a.get_text()
                scl_score2[2] = school.span.get_text()
    elem_sold2.append(scl2[0])
    mid_sold2.append(scl2[1])
    high_sold2.append(scl2[2])
    elem_s_sold2.append(scl_score2[0])
    mid_s_sold2.append(scl_score2[1])
    high_s_sold2.append(scl_score2[2])


['None', 'None', 'None', 'None', 'None', 'None', 'None']
['None', 'None', 'None', 'None', 'None', 'None', 'None']
['Single Family', '1963', 'Forced air, Wall', 'Central, Wall', '2 spaces', 'No Data', '6,534 sqft', '4', 'Stall Shower, Solid Surface', 'Shower Over Tub, Solid Surface', 'Dishwasher, Garbage Disposal, Range/Oven Free Standing, Counter - Solid Surface, Updated Kitchen, Breakfast Bar, Island, Skylight(s)', 'Dining Area, Bonus/Plus Room', 'In Garage, Hookups Only', '4 Bedrooms, 3 Baths', 'Forced air, Wall', 'Electric, Gas', 'Central, Wall', 'Forced Air 1 Zone, Wall Furnace, Electric, Gas', 'Ceiling Fan(s), No Air Conditioning, Window/Wall Unit(s)', 'Dishwasher, Garbage disposal, Range / Oven', '2,122 sqft', 'Carpet, Laminate, Tile', 'Hardwood Floors, Laminate, Tile, Carpet', '988', '8', 'Garage Door Opener, Window Coverings, Tankless Water Heater', '2', 'One Story', '1', 'Other', 'Single Family', 'RESIDENTIAL', 'Ranch', 'Composition', 'Stucco, Wood', 'Stucco, Wood Siding', 'Co

['Single Family', '1998', 'Forced air', 'Central', 'Attached Garage', '$75/month', '5,937 sqft', '5', 'Ground Floor Bedroom, Master Suite / Retreat, Walk-in Closet', '3 full, 1 half', 'Showers over Tubs - 2+, Double Sinks, Marble, Master - Stall Shower(s)', 'Garbage Disposal, Pantry, Countertop - Granite, Island, Hood Over Range', 'Breakfast Nook, Dining Area in Living Room, Dining Bar', 'Separate Family Room, Kitchen / Family Room Combo', 'Laundry Room', 'In Utility Room', 'Forced air', 'Central', 'Central Forced Air', 'Central AC', 'Garbage disposal', '2,850 sqft', 'Carpet, Laminate, Tile', 'Tile, Laminate, Carpet', '9', 'Family Room', 'High Ceiling, Bay Window, Walk-in Closet', '1', 'Contemporary', 'Single Family', 'Detached', 'Residential', 'Contemporary', 'Exclusive Right to Sell', 'Tile', 'Concrete Slab', 'Tile', 'Brick (Reinforced), Stucco', '1998', 'Built in 1998', '0', '5,937 sqft', 'Square Feet', 'Square Feet', 'Regular', '51373929', 'Attached Garage', 'Attached Garage', 'Pub

['None', 'None', 'None', 'None', 'None', 'None', 'None']
['Single Family', '1977', 'Forced air', 'None', '2 spaces', 'No Data', '8,001 sqft', '4', 'Tub, Solid Surface, Updated Baths, Shower Over Tub', 'Shower Over Tub, Updated Baths, Solid Surface', 'Dishwasher, Microwave, Counter - Solid Surface, Gas Range/Cooktop, Refrigerator, Updated Kitchen, Island, Counter - Stone', '1 Bath, 1 Bedroom, Main Entry', 'Master Bedrm Suite - 1, 2 Bedrooms, 1 Bedroom', 'Dining Area, Family Room, Kitchen/Family Combo', 'Hookups Only', 'Forced air', 'Gas', 'None', 'Forced Air 1 Zone', 'Dishwasher, Microwave, Range / Oven, Refrigerator', '2,186 sqft', 'Laminate', 'Laminate', 'Fireplace', '8', 'Water Heater Gas', '1', 'Two Story', '1', 'Barbecue Area', 'Wet Bar', 'Contemporary', 'Single Family', 'RESIDENTIAL', 'Traditional', 'Tile', 'Stucco', 'Stucco', 'Tile', 'Existing', '1979', 'Built in 1977', '2', 'Deck', 'Patio', 'Fenced, Front Yard, Side Yard, Tool Shed, Patio Enclosed', 'Territorial', 'Hills', '8,00

['Single Family', '1964', 'Forced air', 'Central', '2 spaces', 'No Data', '9,970 sqft', '4', 'Stall Shower, Solid Surface, Tile, Updated Baths', 'Shower Over Tub, Updated Baths, Solid Surface, Stall Shower', 'Dishwasher, Electric Range/Cooktop, Double Oven, Range/Oven Built-in, Updated Kitchen, Counter - Stone, Oven Built-in', '1 Bath, Laundry Facility, 1 Bedroom, Main Entry', '3 Bedrooms, 2 Baths, Master Bedrm Suite - 1', 'Dining Area, Family Room, Formal Dining Room', 'In Garage', 'Forced air', 'Central', 'Forced Air 1 Zone', 'Ceiling Fan(s), No Air Conditioning', 'Dishwasher, Range / Oven', '2,160 sqft', 'Carpet, Linoleum / Vinyl, Tile', 'Linoleum, Tile, Carpet', 'Fireplace', 'Ceiling Fan', '8', 'Mirrored Closet Door(s)', '1', 'Two Story', '1', 'Other', 'Single Family', 'RESIDENTIAL', 'Other', 'Composition', 'Brick, Stucco, Wood', 'Stucco, Wood Siding, Brick', 'Composition Shingles', 'Existing', '1966', 'Built in 1964', '0', 'Deck', 'Patio', 'Lawn', 'Fenced Yard', 'Garden', 'Deck(s)

['Single Family', '1954', 'Forced air', 'No Data', 'Attached Garage', 'No Data', '6,716 sqft', '4', 'Shower Over Tub, Solid Surface', 'Solid Surface, Shower Over Tub', 'Dishwasher, Eat In Kitchen, Gas Range/Cooktop, Refrigerator, Ice Maker Hookup, Updated Kitchen, Breakfast Bar, Island, Counter - Stone, Self-Cleaning Oven', 'Laundry Facility, 3 Bedrooms, Main Entry, 2 Baths, Master Bedrm Suite - 1, No Steps to Entry', 'Dining Area, Family Room, Kitchen/Family Combo', 'In Garage, Dryer, Washer', '1 Bath, 1 Bedroom, Loft, Master Bedrm Suites - 2', 'Forced air', 'Gas', 'Forced Air 1 Zone', 'Ceiling Fan(s)', 'Dishwasher, Dryer, Range / Oven, Refrigerator, Washer', '1,900 sqft', 'Carpet, Laminate', 'Laminate, Carpet', '8', 'Water Heater Gas, Washer, Dryer', '0', 'Two Story', '1', 'Contemporary', 'Single Family', 'RESIDENTIAL', 'Contemporary', 'Composition', 'Stucco', 'Stucco, Dual Pane Windows', 'Composition Shingles', 'Existing', '1984', 'Built in 1954', '0', 'Patio, Back Yard, Fenced, Dog

In [20]:
len(lotsize_sold2)

40

In [21]:
lotsize_sold=lotsize_sold1+lotsize_sold2
year_sold=year_sold1+year_sold2
ac_sold=ac_sold1+ac_sold2
heat_sold=heat_sold1+heat_sold2
park_sold=park_sold1+park_sold2
hoa_sold=hoa_sold1+hoa_sold2
elem_sold=elem_sold1+elem_sold2
mid_sold=mid_sold1+mid_sold2
high_sold=high_sold1+high_sold2
elem_s_sold=elem_s_sold1+elem_s_sold2
mid_s_sold=mid_s_sold1+mid_s_sold2
high_s_sold=high_s_sold1+high_s_sold2
neighborhood_sold=neighborhood_sold1+neighborhood_sold2

In [22]:
#add a columns to the empty dataframe
soldhouse_df['bds']=bds_sold
soldhouse_df['bath']=bath_sold
soldhouse_df['areas']=areas_sold
soldhouse_df['lot_size']=lotsize_sold
soldhouse_df['year_built']=year_sold
soldhouse_df['a/c']=ac_sold
soldhouse_df['heater']=heat_sold
soldhouse_df['garage']=park_sold
soldhouse_df['elem_score']=elem_s_sold
soldhouse_df['middle_score']=mid_s_sold
soldhouse_df['high_score']=high_s_sold
soldhouse_df['elem_school'] = elem_sold
soldhouse_df['middle_school'] = mid_sold
soldhouse_df['high_school'] = high_sold
soldhouse_df['sold_price'] = price_ls_sold
soldhouse_df['HOA'] = hoa_sold
soldhouse_df['address']=adr_ls_sold
soldhouse_df['date_sold']=sold_dates

In [23]:
len(bath_sold)

80

In [63]:
#save data on excel file
with pd.ExcelWriter('zillow_data.xlsx') as writer:  
    house_df.to_excel(writer, sheet_name='on_market_data')
    soldhouse_df.to_excel(writer, sheet_name='sold_data')
writer.save()

In [66]:
market_df = pd.read_excel('zillow_data.xlsx', sheet_name='on_market_data', index_col=[0])

In [67]:
market_df.drop(35, inplace=True)
market_df.bds = market_df.bds.apply(lambda x: x.split(' ')[0]).astype(int)
market_df.bath = market_df.bath.apply(lambda x: x.split(' ')[0]).astype(float)
market_df.areas = market_df.areas.apply(lambda x: x.split(' ')[0].replace(",", "")).astype(float)
market_df.lot_size = market_df.lot_size.apply(lambda x: x.split(' ')[0].replace(",", "")).astype(float)
market_df.lot_size = market_df.lot_size.apply(lambda x: round(x*43560,2) if x < 100 else x)#it has acre value in it
market_df.rename(columns={'a/c': 'central_ac'}, inplace=True)
market_df.central_ac = market_df.central_ac.apply(lambda x: 1 if x == 'Central' else 0)
market_df.rename(columns={'heater': 'central_heat'}, inplace=True)
market_df.central_heat = market_df.central_heat.apply(lambda x: 1 if 'Forced air' in x else 0)
market_df.price = market_df.price.str[1:].apply(lambda x: x.replace(",", "")).astype(float)
market_df.rename(columns={'price/sqft': 'price_per_sqft'}, inplace=True)
price_38 = '$'+(market_df.price[38]/market_df.areas[38]).round(2).astype(str)
market_df.loc[38, 'price_per_sqft']= price_38
market_df.price_per_sqft=market_df.price_per_sqft.apply(lambda x: x.split('$')[1].replace(",", "")).astype(float)
market_df.HOA = market_df.HOA.apply(lambda x: 0 if x == 'No Data' or x == 'None' else float(re.findall('\d+', x)[0]))

In [69]:
school_df = pd.concat([market_df['elem_school'], market_df['middle_school'], market_df['high_school']], axis=1)

In [70]:
school_df.groupby(['middle_school','high_school', 'elem_school'])['high_school'].count()

middle_school                       high_school                   elem_school                        
Centerville Junior High             Washington High School        Niles Elementary School                4
                                                                  Parkmont Elementary School             3
                                                                  Vallejo Mill Elementary School         2
G. M. Walters Junior High School    John F. Kennedy High School   Brier Elementary School                2
                                                                  J. Haley Durham Elementary School      1
                                                                  John Blacow Elementary School          1
                                                                  John G. Mattos Elementary School       1
John M. Horner Junior High School   Irvington High School         Harvey Green Elementary School         1
                                          

##### From school information we come to a conclusion that one middle school corresponding to one high school and many elementary school. so we need to find out what elementary school for the null value elementary school is.

In [107]:
market_df[market_df['elem_school']=='None'].address

Series([], Name: address, dtype: object)

In [105]:
market_df[market_df['middle_school']=='None'].address

Series([], Name: address, dtype: object)

In [106]:
market_df[market_df['high_school']=='None'].address

Series([], Name: address, dtype: object)

After checking 3 address on the 'www.myschoollocation.com'. We found that all 4 address has the same elementary schools, Leitch elementary school (k-2) and Warm Springs Elementary School (3-6).
Leitch elementary school (k-2) A in niche, equal 9 score in great school rating.
Warm Springs Elementary School (3-6) 9 score in great school rating, so the avg score for k-6 in warm spring is 9. 
so we can replace the None value with 9.

In [72]:
market_df['elem_school'].replace('None', 'Leitch elementary school + Warm Springs Elementary School', inplace=True)
market_df['elem_score'].replace('None', 9, inplace=True)

In [73]:
market_df[['elem_score', 'middle_score', 'high_score']]=market_df[['elem_score', 'middle_score', 'high_score']].astype(int)

In [74]:
market_df.garage.value_counts()

2 spaces           23
Attached Garage     6
3 spaces            4
4 spaces            3
1 space             2
Detached Garage     1
Name: garage, dtype: int64

In [75]:
market_df.garage.replace('No Data', '0', inplace=True)

In [76]:
market_df.garage = market_df.garage.apply(lambda x: 2 if len(re.findall('\d', x))==0 else int(re.findall('\d', x)[0])) 
#replace attached and detached garage with park spaces 2 spaces(most frequent).

In [77]:
sold_df = pd.read_excel('zillow_data.xlsx', sheet_name='sold_data', index_col=[0])

In [79]:
sold_df.bds = sold_df.bds.apply(lambda x: 0 if x == 'Studio' else x.split(' ')[0]).astype(int)
#after checked this house, we found the bath number is 3, so replace the none value with 3
sold_df.bath.replace('-- ba', '3 ba', inplace=True)
sold_df.bath = sold_df.bath.apply(lambda x: x.split(' ')[0]).astype(float)
sold_df.areas = sold_df.areas.apply(lambda x: x.split(' ')[0].replace(",", "")).astype(float)
#too many null value in this row, so we should just delete it
sold_df.drop(sold_df[sold_df.lot_size=='None'].index, inplace=True)

In [80]:
sold_df[sold_df.lot_size=='No Data']

,bds,bath,areas,lot_size,year_built,a/c,heater,garage,elem_score,middle_score,high_score,elem_school,middle_school,high_school,sold_price,HOA,address,date_sold
23,4,3.0,2105.0,No Data,2015,Central,Forced air,Attached Garage,9,10,10,John Gomes Elementary School,William Hopkins Junior High School,Mission San Jose High School,$1.38M,$234/month,"2839 Casita Ter, Fremont, CA 94539",03/03/2020


In [84]:
sold_df.loc[23, 'lot_size'] = '2,105 sqft' # checked on zillow, replaced no data with floor size

In [85]:
sold_df.lot_size = sold_df.lot_size.apply(lambda x: x.split(' ')[0].replace(",", "")).astype(float)
sold_df.lot_size = sold_df.lot_size.apply(lambda x: round(x*43560,2) if x < 100 else x)#it has acre value in it
sold_df.rename(columns={'a/c': 'central_ac'}, inplace=True)
sold_df.central_ac = sold_df.central_ac.apply(lambda x: 1 if x == 'Central' else 0)
sold_df.rename(columns={'heater': 'central_heat'}, inplace=True)
sold_df.central_heat = sold_df.central_heat.apply(lambda x: 1 if 'Forced air' in x else 0)
sold_df.sold_price = sold_df.sold_price.str[1:].apply(lambda x: float(re.findall('(\d+[.,]\d+)M$', x)[0])*1000000 if 'M' in x else float(x.replace(',','')))

In [86]:
sold_df.HOA = sold_df.HOA.apply(lambda x: 0 if x == 'No Data' or x == 'None' else float(re.findall('\d+', x)[0]))

In [87]:
sold_df.garage.replace('No Data', '0', inplace=True)

In [88]:
sold_df.garage.value_counts()
sold_df.garage = sold_df.garage.apply(lambda x: 2 if len(re.findall('\d', x))==0 else int(re.findall('\d', x)[0]))

In [97]:
datetime.strptime(sold_df.date_sold[0], '%m/%d/%Y').date()

datetime.date(2020, 3, 11)

In [99]:
#find out where the date time format is not correct.
for (i, x) in enumerate(sold_df.date_sold):
    try:
        i, datetime.strptime(x, '%m/%d/%Y').date()
    except:
        print(i, "An exception occurred")

In [102]:
sold_df.date_sold = sold_df.date_sold.apply(lambda x: datetime.strptime(x, '%m/%d/%Y').date())

In [110]:
sold_df[sold_df['elem_school']=='None'].address

12    872 Lippert Ave, Fremont, CA 94539
14    155 Camphor Ave, Fremont, CA 94539
Name: address, dtype: object

After checking 2 address on the 'www.myschoollocation.com'. We found that all 2 address has the same elementary schools, Leitch elementary school (k-2) and Warm Springs Elementary School (3-6). Leitch elementary school (k-2) A in niche, equal 9 score in great school rating. Warm Springs Elementary School (3-6) 9 score in great school rating, so the avg score for k-6 in warm spring is 9. so we can replace the None value with 9.

In [111]:
sold_df['elem_school'].replace('None', 'Leitch elementary school + Warm Springs Elementary School', inplace=True)
sold_df['elem_score'].replace('None', 9, inplace=True)

In [112]:
sold_df[['elem_score', 'middle_score', 'high_score']]=sold_df[['elem_score', 'middle_score', 'high_score']].astype(int)

In [114]:
#save data on excel file
with pd.ExcelWriter('cleaned_data.xlsx') as writer:  
    market_df.to_excel(writer, sheet_name='on_market_data')
    sold_df.to_excel(writer, sheet_name='sold_data')
writer.save()

In [ ]:
import boto3

# Create an S3 client
s3 = boto3.client('s3')

# Specify the file to upload
filename = 'path/to/your/file'
bucket_name = 'your-bucket-name'
s3_key = 'your-desired-s3-object-key'  # This is the target file name in S3

# Upload the file
s3.upload_file(filename, bucket_name, s3_key)
